### Created a Neural Network class. This class takes in list of Layer objects. It also takes a loss object.You can also allow it to take a seed as input. The seed is used for reproduciblity across runs. Each layer is characterized by its activation function and count of output neuron.

In [208]:
import numpy as np
import matplotlib.pyplot as plt

def linear(x):
    return x

def linear_activation_derivative(x):
    return 1

def sigmoid(x):
    # Clip x to prevent overflow.
    x_clipped = np.clip(x, -500, 500)
    return 1 / (1 + np.exp(-x_clipped))

def sigmoid_derivative(x):
    s = sigmoid(x)
    return s * (1 - s)

def tanh(x):
    return np.tanh(x)

def tanh_derivative(x):
    return 1 - np.tanh(x) ** 2

def mse_loss(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

def mse_loss_derivative(y_true, y_pred):
    return 2 * (y_pred - y_true) / y_true.size

def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return (x > 0).astype(float)

def softmax(x):
    return np.exp(x) / sum(np.exp(x))

def cross_entropy_loss(y_true, y_pred):
    num_samples = y_true.shape[0]
    loss = -np.sum(y_true * np.log(y_pred + 1e-9)) / num_samples
    return loss

def softmax_derivative(x):
  return np.exp(x) / sum(np.exp(x)) * (1. - np.exp(x) / sum(np.exp(x)))


def cross_entropy_loss_derivative(y_true, y_pred):
    num_samples = y_true.shape[0]
    return (y_pred - y_true) / num_samples

def get_accuracy(Y_hat, Y):
    # Convert one-hot encoded predictions to class labels
    y_pred_labels = np.argmax(Y_hat, axis=1)
    # Convert one-hot encoded true labels to class labels
    y_true_labels = np.argmax(Y, axis=1)
    # Calculate accuracy by comparing predicted labels with true labels
    accuracy = (y_pred_labels == y_true_labels).mean()*100
    return accuracy

def get_predictions(AL):
    # Step 1: Find the index of the max probability in each row
    max_indices = np.argmax(AL, axis=1)

    # Step 2: Create a new matrix of zeros with the shape (number of rows in AL, number of classes)
    # Assuming number of classes is equal to the number of rows in AL for a single sample scenario
    one_hot_predictions = np.zeros_like(AL)

    # Step 3: Set the elements to 1 at each row's max index
    # np.arange(AL.shape[0]) creates an array of row indices, max_indices specifies the column for each row
    one_hot_predictions[np.arange(AL.shape[0]), max_indices] = 1

    return one_hot_predictions


In [209]:
class Layer:
    def __init__(self, input_size, output_size, activation_function, activation_derivative):
        self.weights = np.random.randn(input_size, output_size) * 0.1  # Corrected dimensions
        self.biases = np.random.rand() * 10
        self.activation_function = activation_function
        self.activation_derivative = activation_derivative
        self.output = None
        self.input = None
        self.activation_input = None
        self.d_weights = None
        self.d_biases = None

    def forward(self, input_data):
        self.input = input_data
        # Correctly apply dot product given the input and weight dimensions
        self.activation_input = np.dot(input_data, self.weights) + self.biases
        self.output = self.activation_function(self.activation_input)
        return self.output

    def backward(self, output_error, learning_rate):
        d_activation_input = output_error * self.activation_derivative(self.activation_input)

        # Correct gradient calculation based on the updated dot product order
        self.d_weights = np.dot(self.input.T, d_activation_input)
        self.d_biases = np.sum(d_activation_input, axis=0, keepdims=True)  # Corrected axis for biases

        # Correct backpropagated error calculation
        input_error = np.dot(d_activation_input, self.weights.T)

        # Update weights and biases
        self.weights -= learning_rate * self.d_weights
        self.biases -= learning_rate * self.d_biases

        return input_error


In [210]:
class NeuralNetwork:
    def __init__(self, layers, loss, loss_derivative, seed=None):
        np.random.seed(seed)
        self.layers = layers
        self.loss = loss
        self.loss_value=None
        self.loss_derivative = loss_derivative

    def forward(self, input_data):
        for layer in self.layers:
            input_data = layer.forward(input_data)
        return input_data

    def backward(self, y_true, learning_rate):
            """
            Perform backward propagation through the neural network.
            """
            output_error = self.loss_derivative(y_true, self.layers[-1].output)
            input_error = output_error
            for layer in reversed(self.layers):
                input_error = layer.backward(input_error, learning_rate)
            return input_error
    def train_sgd(self, x_train, y_train, learning_rate, epochs, batch_size):
        n_samples = x_train.shape[0]
        
        for epoch in range(epochs):
            # Shuffle the dataset at the beginning of each epoch
            indices = np.arange(n_samples)
            np.random.shuffle(indices)
            x_train_shuffled = x_train[indices]
            y_train_shuffled = y_train[indices]
            
            # Stochastic Gradient iteration
            for start_idx in range(0, n_samples, batch_size):
                end_idx = min(start_idx + batch_size, n_samples)
                x_batch = x_train_shuffled[start_idx:end_idx]
                y_batch = y_train_shuffled[start_idx:end_idx]
                
                # Forward and backward passes for the Stochastic 
                self.forward(x_batch)
                self.backward(y_batch, learning_rate)
            
            
            output = self.forward(x_train)
            epoch_loss = self.loss(y_train, output)
            # print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss}")
    
    # def train(self, x_train, y_train, learning_rate):

    # # Assuming x_train is a 2D array where each row is a sample
    # # and y_train is a 2D array where each row is a target value or vector
    #     loss=0;

    #     # Reshape the i-th sample to a column vector
    #     # x_sample = x_train
    #     # y_sample = y_train

    #     # Forward propagation
    #     output = self.forward(x_train)

    #     # Compute loss
    #     loss = self.loss(y_train, output)
    #     self.loss_value=loss

    #     # Backward propagation
    #     self.backward(y_train, learning_rate)


    #     # print(f"Iteration: Loss: {loss}")





### 1. Just one output neural with linear activation and least mean square loss.(This is linear regression).


In [ ]:


from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

# Fetch the California Housing dataset
california_housing = fetch_california_housing()
X = california_housing.data
Y = california_housing.target.reshape(-1, 1)  # Ensure Y is a column vector

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42)

print(X_train.shape)
print(Y_train.shape)

# Assuming your classes and functions are defined as in the provided code snippet

# Initialize layers
input_size = X_train.shape[1]  # Number of features in the dataset
layer1 = Layer(input_size=input_size, output_size=1 ,activation_function=linear, activation_derivative=linear_activation_derivative)

# Initialize the neural network
nn1 = NeuralNetwork(layers=[layer1], loss=mse_loss, loss_derivative=mse_loss_derivative, seed=42)
# Training settings
epochs = 200 # Number of epochs to train
learning_rate = 0.000001# Learning rate
losses = []
for epoch in range(epochs):
    
        # Train the neural network on the sample
    nn1.train_sgd(X_train, Y_train, learning_rate,50,100)
    if epoch%1==0:
      output1 = nn1.forward(X_train)
      loss1 = mse_loss(Y_train,output1)
      losses.append(loss1)
      print(f"Epoch {epoch+1}/{epochs} {loss1}")
      

output1 = nn1.forward(X_train)
loss1 = mse_loss(Y_train,output1)
print("loss1 train :",loss1)
output1 = nn1.forward(X_test)
loss1 = mse_loss(Y_test,output1)
print("loss1 test :",loss1)

plt.plot(losses,color="green")
plt.xlabel("Iteration count")
plt.ylabel("Loss of Neural Network 2.1")
plt.show()



### 2. Two layers. Layer 1 with 13 output neurons with sigmoid activation. Layer 2 with one output neuron and linear activation. use mean squared loss

In [ ]:
layer1 = Layer(input_size=input_size, output_size=13 ,activation_function=sigmoid, activation_derivative=sigmoid_derivative)
layer2 = Layer(input_size=13, output_size=1 ,activation_function=linear, activation_derivative=linear_activation_derivative)
nn2 = NeuralNetwork(layers=[layer1,layer2], loss=mse_loss, loss_derivative=mse_loss_derivative, seed=42)
epochs = 200  # Number of epochs to train
learning_rate = 0.001 # Learning rate
losses = []
for epoch in range(epochs):

        # Train the neural network on the sample
    nn2.train_sgd(X_train, Y_train, learning_rate,50,100)
    if epoch%1==0:
        output2 = nn2.forward(X_train)
        loss2 = mse_loss(Y_train,output2)
        losses.append(loss2)
        print(f"Epoch {epoch+1}/{epochs} {loss2}")




output2 = nn2.forward(X_train)
loss2 = mse_loss(Y_train,output2)
print("loss2 train :",loss2)
output2 = nn2.forward(X_test)
loss2 = mse_loss(Y_test,output2)
print("loss1 test:",loss2)

plt.plot(losses,color="green")
plt.xlabel("Iteration count")
plt.ylabel("Loss of Neural Network 2.2")
plt.show()

### 3. Three layers. Layer 1 with 13 output neurons with sigmoid activation.Layer 2 with 13 output neurons and sigmoid activation. Layer 3 with one output neuron and linear activation. use mean squared loss

In [ ]:
layer1 = Layer(input_size=input_size, output_size=13 ,activation_function=sigmoid, activation_derivative=sigmoid_derivative)
layer2 = Layer(input_size=13, output_size=13 ,activation_function=sigmoid, activation_derivative=sigmoid_derivative)
layer3 = Layer(input_size=13, output_size=1 ,activation_function=linear, activation_derivative=linear_activation_derivative)
nn3 = NeuralNetwork(layers=[layer1,layer2,layer3], loss=mse_loss, loss_derivative=mse_loss_derivative, seed=42)
epochs = 100  # Number of epochs to train
learning_rate = 0.01# Learning rate
losses = []
for epoch in range(epochs):

        # Train the neural network on the sample
    nn3.train_sgd(X_train, Y_train, learning_rate,50,100)
    if epoch%1==0:
        output3 = nn3.forward(X_train)
        loss3 = mse_loss(Y_train,output3)
        losses.append(loss3)
        print(f"Epoch {epoch+1}/{epochs} {loss3}")
output3 = nn3.forward(X_train)
loss3 = mse_loss(Y_train,output3)
print("loss3 train:",loss3)
output3 = nn3.forward(X_test)
loss3 = mse_loss(Y_test,output3)
print("loss3 test:",loss3)

plt.plot(losses,color="green")
plt.xlabel("Iteration count")
plt.ylabel("Loss of Neural Network 2.3")
plt.show()


### 1. Two layers Layer 1 with 89 output neurons with tanh activation. Layer 2 with ten output neuron and sigmoid activation. use mean squared loss


In [ ]:
# from sklearn.datasets import fetch_openml
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler, OneHotEncoder
# import numpy as np

# # Fetch the MNIST dataset
# mnist = fetch_openml('mnist_784', version=1,as_frame=False)
# X = mnist.data
# Y = mnist.target.astype('int32')  # Convert target to integer

# # Normalize features
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# # One-hot encode the labels
# encoder = OneHotEncoder(categories='auto')
# Y_one_hot = encoder.fit_transform(Y.reshape(-1, 1)).toarray()

# # Split the dataset into training and test sets
# X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y_one_hot, test_size=0.2, random_state=42)

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import numpy as np

# Load the digits dataset
digits = load_digits()
# Extract features (X) and target labels (Y)
X = digits.data
Y = digits.target
shuffle_indices = np.random.permutation(len(X))
X,Y= X[shuffle_indices], Y[shuffle_indices]

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# One-hot encode the labels (not necessary for this classification task)
encoder = OneHotEncoder(categories='auto')
Y_one_hot = encoder.fit_transform(Y.reshape(-1, 1)).toarray()

# Split the dataset into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y_one_hot, test_size=0.2, random_state=42)
print(X_train.shape)
print(Y_train.shape)
# print(X[0])
# print(Y_one_hot)
input_size = X_train.shape[1]  # Number of features in the dataset
layer1 = Layer(input_size=input_size, output_size=89 ,activation_function=tanh, activation_derivative=tanh_derivative)
layer2 = Layer(input_size=89, output_size=10 ,activation_function=sigmoid, activation_derivative=sigmoid_derivative)

# Initialize the neural network
nn_digits = NeuralNetwork(layers=[layer1, layer2], loss=mse_loss, loss_derivative=mse_loss_derivative, seed=42)
# Training settings
epochs = 100 # Number of epochs to train
learning_rate = 0.5# Learning rate
accuracies = []
for epoch in range(epochs):
    nn_digits.train_sgd(X_train, Y_train, learning_rate,50,100)
    y_hat1 = nn_digits.forward(X_train)
    # print(y_hat)
    y_hat = get_predictions(y_hat1)
    # print(y_hat)
    if epoch%1==0:
      accuracies.append(get_accuracy(y_hat,Y_train))
      print(f"Epoch {epoch+1}/{epochs} {get_accuracy(y_hat,Y_train)}")

output_digits_train = nn_digits.forward(X_train)
y_hat_digits_train = get_predictions(output_digits_train)
accuracy = get_accuracy(y_hat_digits_train,Y_train)
print("Accuracy digits train :",accuracy)

output_digit_test = nn_digits.forward(X_test)
y_hat_digit_test = get_predictions(output_digit_test)
accuracy = get_accuracy(y_hat_digit_test,Y_test)
print("Accuracy digits test :",accuracy)

plt.plot(accuracies,color="green")
plt.xlabel("Iteration count")
plt.ylabel("Accuracy of Neural Network 3.1")
plt.show()

### 2. Two layers. Layer 1 with 89 output neurons with tanh activation. Layer 2 with ten output neuron and linear activation. use softmax with cross entropy loss.

In [204]:
input_size = X_train.shape[1]  # Number of features in the dataset
layer1 = Layer(input_size=input_size, output_size=89 ,activation_function=tanh, activation_derivative=tanh_derivative)
layer2 = Layer(input_size=89, output_size=10 ,activation_function=softmax, activation_derivative=softmax_derivative)
nn_digit2 = NeuralNetwork(layers=[layer1, layer2], loss=cross_entropy_loss, loss_derivative=cross_entropy_loss_derivative, seed=42)


In [ ]:

# Training settings
epochs = 60  # Number of epochs to train
learning_rate = 0.005 # Learning rate
accuracies = []
for epoch in range(epochs):
    nn_digit2.train_sgd(X_train, Y_train, learning_rate,50,100)
    y_hat1 = nn_digit2.forward(X_train)
    # print(y_hat)
    y_hat = get_predictions(y_hat1)
    # print(y_hat)
    if epoch%1==0:
      accuracies.append(get_accuracy(y_hat,Y_train))
      print(f"Epoch {epoch+1}/{epochs} {get_accuracy(y_hat,Y_train)}")


output_digits_train = nn_digit2.forward(X_train)
y_hat_digits_train = get_predictions(output_digits_train)
accuracy = get_accuracy(y_hat_digits_train,Y_train)
print("Accuracy digit train :",accuracy)

output_digit_test = nn_digit2.forward(X_test)
y_hat_digit_test = get_predictions(output_digit_test)
accuracy = get_accuracy(y_hat_digit_test,Y_test)
print("Accuracy digit test :",accuracy)

plt.plot(accuracies,color="green")
plt.xlabel("Iteration count")
plt.ylabel("Accuracy of Neural Network 3.2")
plt.show()

In [1]:
from sklearn.datasets import load_iris, load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer, OneHotEncoder

In [ ]:
class Layer:
    def __init__(self, input_size, output_size, activation_function, activation_derivative):
        self.weights = np.random.randn(input_size, output_size) * np.sqrt(1 / (input_size + output_size))  # Corrected dimensions
        self.biases = 0
        self.activation_function = activation_function
        self.activation_derivative = activation_derivative
        self.output = None
        self.input = None
        self.activation_input = None
        self.d_weights = None
        self.d_biases = None

    def forward(self, input_data):
        self.input = input_data
        # Correctly apply dot product given the input and weight dimensions
        self.activation_input = np.dot(self.weights.T,input_data) + self.biases
        self.output = self.activation_function(self.activation_input)
        return self.output

    def backward(self, output_error, learning_rate):
        # print(output_error.shape)
        # print(self.activation_derivative(self.activation_input).shape)
        d_activation_input = output_error * self.activation_derivative(self.activation_input)
        #print(d_activation_input.shape)
        # Correct gradient calculation based on the updated dot product order
        self.d_weights = np.dot(self.input, d_activation_input.T)
        self.d_biases = np.sum(d_activation_input, axis=0, keepdims=True)  # Corrected axis for biases

        # Correct backpropagated error calculation
        input_error = np.dot(self.weights,d_activation_input)

        # Update weights and biases
        self.weights -= learning_rate * self.d_weights
        self.biases -= learning_rate * self.d_biases

        return input_error


In [ ]:
class NeuralNetwork:
    def __init__(self, layers, loss, loss_derivative, seed=None):
        np.random.seed(seed)
        self.layers = layers
        self.loss = loss
        self.loss_value=None
        self.loss_derivative = loss_derivative

    def forward(self, input_data):
        for layer in self.layers:
            input_data = layer.forward(input_data)
        return input_data

    def backward(self, y_true, learning_rate):
            """
            Perform backward propagation through the neural network.

            Args:
            y_true (numpy.ndarray): True labels.
            learning_rate (float): Learning rate for updating weights and biases.

            Returns:
            numpy.ndarray: Gradient of loss with respect to the input of the first layer.
            """
            output_error = self.loss_derivative(y_true.T, self.layers[-1].output)

            # print(y_true.shape)
            # print(self.layers[-1].output.shape)
            # print("out:",output_error.shape)
            input_error = output_error
            for layer in reversed(self.layers):
                input_error = layer.backward(input_error, learning_rate)
            return input_error
    def train(self, x_train, y_train, learning_rate):

    # Assuming x_train is a 2D array where each row is a sample
    # and y_train is a 2D array where each row is a target value or vector
        loss=0

        # Reshape the i-th sample to a column vector
        # x_sample = x_train
        # y_sample = y_train

        # Forward propagation
        output = self.forward(x_train)

        # Compute loss
        loss = self.loss(y_train, output)
        self.loss_value=loss

        # Backward propagation
        self.backward(y_train, learning_rate)


        # print(f"Iteration: Loss: {loss}")




In [ ]:
import numpy as np

class Conv_Layer:
    def __init__(self, input_shape, output_channels, kernel_size, stride, padding,alpha):
        self.inp_shape = input_shape
        self.input_channel = input_shape[0]
        self.inp_height = input_shape[1]
        self.inp_width = input_shape[2]

        self.output_channels = output_channels
        self.output_shape = (output_channels,self.inp_height,self.inp_width)
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = 1
        self.inp = np.random.rand(*input_shape)
        self.out = np.random.rand(*self.output_shape)
        self.kernel_shape = (3,3)
        #self.inp_shape =
        # Initialize weights and biases
        self.filters_shape = (self.output_channels,self.input_channel, *self.kernel_shape)
        self.filters = np.random.random((self.output_channels,self.input_channel, kernel_size, kernel_size))
        self.biases = np.random.rand(*self.output_shape)
        self.alpha = alpha
        # # Cache variables for backward pass
        # self.cache = None


    def conv(self,inp,weights):
      #print(inp.shape[0])
      m = inp.shape[0]+2
      zero_pad_inp = np.zeros((m,m))
      zero_pad_inp[1:inp.shape[0]+1,1:inp.shape[1]+1] = inp

      l = zero_pad_inp.shape[0] - weights.shape[0]+1
      w = zero_pad_inp.shape[1] - weights.shape[1]+1
      output = np.zeros((l,w))

      for i in range(l):
        for j in range(w):
          mat = zero_pad_inp[i:i+weights.shape[0],j:j+weights.shape[1]]
          mat_n = np.multiply(mat,weights)
          output[i,j] = np.sum(mat_n)

      return output


    def forward(self,):
       #print("con_frwrd")
       self.out = np.zeros(self.output_shape)
       for i in range(self.output_channels):
            for j in range(self.input_channel):
                self.out[i]+= self.conv(self.inp[j],self.filters[i,j])

       flatten_out = self.out.reshape(-1,1)
       return flatten_out




        # Convolution operation


    def backward(self,dl_do):
      dl_do = np.reshape(dl_do,self.output_shape)
      self.dl_df = np.zeros(self.filters_shape)
      self.dl_db = np.zeros_like(self.output_shape)

      for i in range(self.output_channels):
        for j in range(self.input_channel):
          self.dl_df[i,j] = self.conv(self.inp[j],dl_do[i])

      self.filters-=self.alpha*self.dl_df




In [ ]:
class CNN:
  def __init__(self,conv_layer,neural_network,X,Y,alpha):
    self.conv_layer = conv_layer
    self.neural_network = neural_network
    self.X=X
    self.Y =Y
    self.loss=_
    self.y_pred=_
    self.alpha=alpha
  def forward_prop(self,):
    #print("frwrd")
    self.conv_layer.inp = self.X
    self.Z = self.conv_layer.forward()
    X = tanh(self.Z)
    self.y_pred = self.neural_network.forward(X)
    #print(self.y_pred)
    self.loss = cross_entropy_loss(self.Y,self.y_pred)

  def backward_pass(self,):
    dl_dtanhx = self.neural_network.backward(self.Y,self.alpha)
    dtanhx_dx = tanh_derivative(self.Z)
    dl_dx = np.dot(dtanhx_dx,dl_dtanhx.T)
    self.conv_layer.backward(dl_dx)



### Generalize this for any number of input and any number of output channel. Implement both forward and backward passes

In [ ]:
def sg_cnn(X_train,Y_train,X_test,Y_test,cnn,iterations,inp_shape,out_shape):
  for i in range(iterations):
    for j in range(X_train.shape[0]):
      X_sample = X_train[j].reshape(inp_shape)
      Y_sample = Y_train[j].reshape(out_shape)
      cnn.X = X_sample
      cnn.Y = Y_sample
      cnn.forward_prop()
      cnn.backward_pass()
    print("Loss at",i,cnn.loss)

  col_num = X_train.shape[0]
  X_train = X_train.reshape(col_num,8,8)
  acc=0
  for i in range(len(X_test)):
    X_new = X_test[i].reshape(inp_shape)
    cnn.X = X_new
    cnn.Y = Y_test[i].reshape(out_shape)
    cnn.forward_prop()
    y_pred = np.argmax(cnn.y_pred)
    y_true = np.argmax(Y_test[i])
    if(y_pred==y_true):
      acc+=1
  print(acc)
  print("Acc : ",(acc/len(Y_test))*100)




In [ ]:
digits = load_digits()
digits['data'] = 1*(digits['data'] >= 8)
X = digits['data']
Y = digits['target'].reshape(-1,1)

encode = OneHotEncoder()
Y = encode.fit_transform(Y).toarray()


X_train,X_test,Y_train,Y_test = train_test_split(X, Y, test_size=0.2)

### Trained this CNN on mnist dataset. Layer 1: Convolution layer with 16 output channels+flatten +tanh activation. Layer 2: 10 output neuron with linear activation. Softmax cross entropy loss 

In [ ]:
# (self, input_shape, output_channels, kernel_size, stride, padding,alpha)
output_channels = 16
input_shape = (1,8,8)
kernel_size = 3
stride = 1
padding = 1
learning_rate = 0.01
conv_layer_obj = Conv_Layer(input_shape,output_channels,kernel_size,stride,padding,learning_rate)

# input_size, output_size, activation_function, activation_derivative
# Layer(input_size=13, output_size=1 ,activation_function=linear, activation_derivative=linear_activation_derivative)
input_size = output_channels*8*8
output_size = 10
layer1_obj = Layer(input_size,output_size,activation_function=softmax,activation_derivative = softmax_derivative)
layers=[layer1_obj]
# layers, loss, loss_derivative, seed=None
neur_net_obj = NeuralNetwork(layers,cross_entropy_loss,cross_entropy_loss_derivative,42)
cnn_obj = CNN(conv_layer_obj,neur_net_obj,X_train,Y_train,0.001)
out_shape = (1,10)
iterations = 100
#print(X_train.shape)
sg_cnn(X_train,Y_train,X_test,Y_test,cnn_obj,iterations,input_shape,out_shape)
